In [ ]:
import matplotlib.pyplot as plt
from slow_control import SlowControlLog

header = "../data/SC_HEADER/1-9-2025"
data = [
    "../data/SC_DATA/4-9-2025",
    "../data/SC_DATA/5-9-2025",
    "../data/SC_DATA/6-9-2025",
    "../data/SC_DATA/7-9-2025",
    "../data/SC_DATA/8-9-2025",
    "../data/SC_DATA/9-9-2025"
]

log = SlowControlLog(header, data)
print(log.available_timerange())    # (Timestamp('2025-09-06 00:00:01'), Timestamp('2025-09-07 23:59:57'))
print(log.list_variables()[:100])    # peek at available channels

# Plot any variable vs time (single-chart, matplotlib)
t_start = "2025-09-04 11:00"
t_end = "2025-09-9 23:59"
# If you want to smooth/noise-reduce:

ax = log.plot(["Pmain","P101","P102", "P103", "P104"], start=t_start, end=t_end, resample="5min")
ax.set_ylabel("Pressure (bar)")
ax.set_ylim(0, 3)  # set y-axis limits if needed


In [ ]:
ax = log.plot(["FM101"], start=t_start, end=t_end, resample="5min")
ax.set_ylabel("Flow (g/min)")

In [ ]:
ax = log.plot(["t0", "t1", "t2", "t3", "t4", "t5", "t6"], start=t_start, end=t_end, resample="5min")
ax.set_ylabel("Temperature (°C)")